## Backtesting Introduction

*Coding along with the Udemy Couse [Machine Learning Applied to Stock & Crypto Trading](https://www.udemy.com/course/machine-learning-applied-to-stock-crypto-trading-python/) by Shaun McDonogh.*

#### Imports

In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt

In [3]:
# Data extraction
start_date = "2017-01-01"
end_date = "2022-06-01"
symbol = "SPY"
df = yf.download(symbol, start=start_date, end=end_date)

[*********************100%***********************]  1 of 1 completed


In [4]:
df.head(3)

Price,Adj Close,Close,High,Low,Open,Volume
Ticker,SPY,SPY,SPY,SPY,SPY,SPY
Date,,,,,,
2017-01-03,197.952881,225.240005,225.830002,223.880005,225.039993,91366500
2017-01-04,199.130569,226.580002,226.750000,225.610001,225.619995,78744400
2017-01-05,198.972351,226.399994,226.580002,225.479996,226.270004,78379000
